In [1]:
import numpy as np
import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression

In [67]:
a = (9.57 - 9.5)/(0.4/np.sqrt(160))
print 2*(1 - scipy.stats.norm.cdf(abs(a)))

0.026856695507523787


In [20]:
data = pd.read_table('diamonds.txt')

In [30]:
x = data.drop(columns = ['price'])
y = pd.DataFrame(data.price)

In [34]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1, shuffle=True)

In [36]:
lr = LinearRegression()
rf = RandomForestRegressor(random_state=1)

In [37]:
lr.fit(x_train, y_train)
rf.fit(x_train, y_train)

/Users/alexey.tyshko/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

In [71]:
otkl_lr = abs(lr.predict(x_test)-y_test)
otkl_rf = abs(rf.predict(x_test).reshape(13485, 1)-y_test)

In [72]:
scipy.stats.ttest_rel(otkl_lr, otkl_rf)

Ttest_relResult(statistic=array([12.93081761]), pvalue=array([5.07417558e-38]))

In [73]:
DescrStatsW(otkl_lr - otkl_rf).tconfint_mean()

(array([73.61008769]), array([99.91403655]))

In [2]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [3]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [4]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [5]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [6]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [7]:
data = pd.read_table('banknotes.txt')

In [8]:
x = data.drop(columns=['real'])
y = data.real

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1, shuffle=True)

In [10]:
l_r1 = LogisticRegression()
l_r2 = LogisticRegression()

In [11]:
lr1 = l_r1.fit(x_train[['X1', 'X2', 'X3']], y_train)
lr2 = l_r2.fit(x_train[['X4', 'X5', 'X6']], y_train)

In [27]:
sample1 = []
for i in range(len(np.array(y_test))):
    if np.array(lr1.predict(x_test[['X1', 'X2', 'X3']]))[i]!=np.array(y_test)[i]:
        sample1.append(1)
    else:
        sample1.append(0)

In [28]:
sample2 = []
for i in range(len(np.array(y_test))):
    if np.array(lr2.predict(x_test[['X4', 'X5', 'X6']]))[i]!=np.array(y_test)[i]:
        sample2.append(1)
    else:
        sample2.append(0)

In [29]:
#sample1 = lr1.predict(x_test[['X1', 'X2', 'X3']])!=y_test
#sample2 = lr2.predict(x_test[['X4', 'X5', 'X6']])!=y_test
print proportions_diff_z_test(proportions_diff_z_stat_rel(sample1, sample2), alternative = 'two-sided')

0.0032969384555543435


In [30]:
print proportions_diff_confint_rel(sample1, sample2, alpha=0.05)

(0.059945206279614305, 0.3000547937203857)


In [16]:
z = (541.4-525.)/(100./np.sqrt(100))
print 1 - stats.norm.cdf(z)

0.05050258347410397


In [17]:
z = (541.5-525.)/(100./np.sqrt(100))
print 1 - stats.norm.cdf(z)

0.0494714680336481
